In [ ]:
#All imports go here
from keras.models import Sequential
from keras.layers import Convolution2D, Cropping2D, Dropout, Flatten,Dense, Lambda, MaxPooling2D
from sklearn.model_selection import train_test_split

import cv2
import numpy as np
import pandas as pd

In [ ]:
# Load data
DATA_NAME = 'f3b3new'
DATA_DIR = './'+DATA_NAME+'/'
LOGFILE_PATH = DATA_DIR + 'driving_log.csv'
IMG_DIR = DATA_DIR + 'IMG/'

logs = pd.read_csv(LOGFILE_PATH,sep=',',header=None)


In [ ]:
#Load steering angles
steering_angles  = np.array(list(logs[3]))

In [ ]:
# Downsample near-zero examples.
eps = 0.3 #Values in the range 0 +/- eps will be pruned
near_zero_idx = np.where(abs(steering_angles) < eps)[0]

delete_prob = 0.95
delete_count = int(delete_prob * len(near_zero_idx))

delete_indices = np.random.choice(near_zero_idx,delete_count,replace=False)


# Update steering angles array.
steering_angles = np.delete(steering_angles,delete_indices)

# Load image data.
img_paths = np.delete(list(logs[0]),delete_indices)

In [ ]:
# # Add steering data for right and left cameras.
# CORRECTION_FACTOR = 0.2 # Steering correction factor for left and right images.
# steering_angles  = np.hstack((steering_angles, steering_angles+CORRECTION_FACTOR, steering_angles-CORRECTION_FACTOR))

# # Load additional image paths from left and right cameras.
# img_paths = np.hstack((img_paths,
#                        np.delete(list(logs[1]),delete_indices),
#                        np.delete(list(logs[2]),delete_indices)))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(steering_angles,bins=np.linspace(-1,1,26))
plt.show()

In [ ]:
# Load image data.
image_data = []

for path in img_paths:
    file_name = path.split('/')[-1]
    img = cv2.cvtColor(cv2.imread(IMG_DIR+file_name),cv2.COLOR_BGR2RGB)
    image_data.append(img)

image_data = np.array(image_data)

In [ ]:
# Augment data by flipping images and steering angles.
image_data_flipped = np.empty_like(image_data)

for i in range(len(image_data)):
    image_data_flipped[i] = np.fliplr(image_data[i])
    
steering_angles_flipped = -steering_angles


image_data = np.vstack((image_data,image_data))
steering_angles = np.hstack((steering_angles,steering_angles_flipped))


In [ ]:
# test-train split
image_data_train, image_data_test, steering_angles_train,steering_angles_test =\
train_test_split(image_data,steering_angles,train_size=0.6)

In [ ]:
# Build and save model.
model = Sequential()
model.add(Lambda(lambda x: x/255 - 0.5,input_shape=image_data.shape[1:]))
model.add(Cropping2D(((70,25),(0,0))))
model.add(Convolution2D(6,5,5,activation='relu'))
#model.add(Dropout(0.3))
model.add(MaxPooling2D())
model.add(Convolution2D(16,5,5,activation='relu'))
#model.add(Dropout(0.3))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1000,activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(400,activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(100,activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.fit(image_data_train, steering_angles_train,  nb_epoch=2, validation_split=0.2, shuffle=True,verbose=2)
print("Test loss: {:.3f}".format(model.evaluate(image_data_test,steering_angles_test,verbose=2)))

model.save('lenet5'+DATA_NAME+'.h5')

In [ ]:
for i in range(10):
    plt.imshow(image_data_test[i])
    plt.title(model.predict(image_data_test[i:i+1],batch_size=1,verbose=2))
    plt.show()

In [ ]:
import os
filenames = os.listdir('./snapshots2')

if not os.path.isdir('./snapshots2-predictions'):
    os.mkdir('./snapshots2-predictions')

for filename in filenames:
    imgdata = cv2.cvtColor(cv2.imread('./snapshots2/'+filename),cv2.COLOR_BGR2RGB)
    prediction = model.predict(imgdata[None,:,:,:],batch_size=1,verbose=2)
    cv2.putText(imgdata,str(prediction),(0,100),cv2.FONT_HERSHEY_PLAIN,1.0,0)
    cv2.imwrite('./snapshots2-predictions/'+filename,imgdata)